In [2]:
#pip install git+https://github.com/tonyo/pyope.git


In [3]:
# 📌 Step 1: Load & Preprocess MNIST Dataset (Plaintext)
from tensorflow.keras.datasets import mnist
from scipy.ndimage import zoom
import numpy as np
from sklearn.model_selection import train_test_split

# Load MNIST dataset (Plaintext)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Print original dataset shapes
print(f"Original Training Data Shape: {X_train.shape}")  # (60,000, 28, 28)
print(f"Original Test Data Shape: {X_test.shape}")  # (10,000, 28, 28)

# ⚡ Increase training set to 80% and test set to 20%
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Updated Training Shape: {X_train.shape}")  # (~56,000, 28, 28)
print(f"Updated Test Shape: {X_test.shape}")  # (~14,000, 28, 28)

# Function to resize images (New size: 20x20 instead of 28x28)
def resize_mnist_batch(images, new_size=20):
    return np.array([zoom(img, new_size / img.shape[0], order=1).flatten() for img in images])

print("Resizing Training Data...")
X_train_resized = resize_mnist_batch(X_train)

print("Resizing Test Data...")
X_test_resized = resize_mnist_batch(X_test)

# Normalize pixel values (0-1)
X_train_resized = X_train_resized / 255.0
X_test_resized = X_test_resized / 255.0

# Print new dataset shapes
print(f"Final Training Data Shape: {X_train_resized.shape}")  # (~56,000, 400)
print(f"Final Test Data Shape: {X_test_resized.shape}")  # (~14,000, 400)


Original Training Data Shape: (60000, 28, 28)
Original Test Data Shape: (10000, 28, 28)
Updated Training Shape: (48000, 28, 28)
Updated Test Shape: (12000, 28, 28)
Resizing Training Data...
Resizing Test Data...
Final Training Data Shape: (48000, 400)
Final Test Data Shape: (12000, 400)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)

# Train the model on plaintext data
print("Training Random Forest on Plaintext Data...")
rf_model.fit(X_train_resized, y_train)

# Perform prediction on plaintext test set
print("Running Inference on Plaintext Data...")
plaintext_predictions = rf_model.predict(X_test_resized)

# Compute accuracy before encryption
accuracy = accuracy_score(y_test, plaintext_predictions)
print(f"🎯 Plaintext Random Forest Accuracy: {accuracy:.4f}")


Training Random Forest on Plaintext Data...
Running Inference on Plaintext Data...
🎯 Plaintext Random Forest Accuracy: 0.9693


In [5]:
import os
import hashlib
import random
from pyope.ope import OPE

# 📌 Step 3.1: Generate Kmaster
def generate_master_key():
    """
    Generates a secure master key for encryption.
    """
    return os.urandom(32)  # Secure 256-bit random key

Kmaster = generate_master_key()
print("✅ Kmaster generated successfully.")

# 📌 Step 3.2: Derive Unique Ktree & Kcomp for Each Tree
def generate_tree_keys(num_trees, master_key):
    """
    Generates unique Ktree and Kcomp keys for each tree using a master key.

    Args:
        num_trees: Number of trees in the Random Forest.
        master_key: The master encryption key (bytes).

    Returns:
        Two lists containing Ktree and Kcomp keys.
    """
    Ktree_list = []
    Kcomp_list = []

    for i in range(num_trees):
        # Generate Ktree key from master key hash
        tree_hash = hashlib.sha512(master_key + f"Tree_{i}".encode()).digest()[:32]
        Ktree_list.append(OPE(tree_hash))  # FIXED: Ensure it's bytes, not hex string

        # Generate Kcomp key uniquely for each tree
        comp_hash = hashlib.sha512(master_key + f"Kcomp_{i}".encode()).digest()[:32]
        Kcomp_list.append(OPE(comp_hash))  # FIXED: Ensure it's bytes, not hex string

    return Ktree_list, Kcomp_list

# ✅ Generate keys for all trees
num_trees = 100
Ktree_list, Kcomp_list = generate_tree_keys(num_trees, Kmaster)

print(f"✅ Derived {num_trees} unique Ktree & Kcomp keys for each tree.")




✅ Kmaster generated successfully.
✅ Derived 100 unique Ktree & Kcomp keys for each tree.


In [6]:
# 📌 Step 4: Encrypt Decision Tree Thresholds using OP-NTRU (Optimized)
import concurrent.futures
from sklearn.tree import _tree
import numpy as np
import random

# 📌 Encrypt a single numerical value using OP-NTRU
def op_ntru_encrypt(value, kcomp, scale_factor=1000):
    """
    Encrypts a numerical value using OP-NTRU with a unique Kcomp.

    Args:
        value: The numerical value (e.g., threshold).
        kcomp: The order-preserving encryption key.
        scale_factor: Scaling factor to prevent loss of precision.

    Returns:
        Encrypted value.
    """
    scaled_value = int(value * scale_factor)  # Scale before encryption
    encrypted_value = kcomp.encrypt(scaled_value)

    # Apply small random noise to avoid identical values
    noise = random.randint(1, 10)  # Small randomness
    return encrypted_value + noise

# 📌 Encrypt thresholds for a single tree
def encrypt_single_tree(tree_idx, tree, Kcomp):
    """
    Encrypts decision thresholds for a single tree.

    Args:
        tree_idx: Index of the tree.
        tree: A single decision tree from RandomForest.
        Kcomp: The encryption key for this tree.

    Returns:
        Encrypted threshold list for this tree.
    """
    tree_thresholds = []
    tree_ = tree.tree_

    for node in range(tree_.node_count):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:  # Ignore leaf nodes
            threshold = tree_.threshold[node]
            encrypted_threshold = op_ntru_encrypt(threshold, Kcomp)
            tree_thresholds.append(encrypted_threshold)
        else:
            tree_thresholds.append(None)  # Maintain alignment

    return tree_thresholds

# 📌 Encrypt thresholds for all trees in the Random Forest with parallel processing
def encrypt_thresholds_parallel(forest, Kcomp_list):
    """
    Encrypts decision thresholds for each tree in the Random Forest using parallel execution.

    Args:
        forest: Trained RandomForestClassifier.
        Kcomp_list: List of Kcomp encryption keys (one per tree).

    Returns:
        A list of encrypted threshold values for each tree.
    """
    total_trees = len(forest.estimators_)
    encrypted_thresholds = [None] * total_trees

    print("🔒 Re-encrypting Decision Tree Thresholds with OP-NTRU (Optimized)...")

    # Use ThreadPoolExecutor to parallelize encryption
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(encrypt_single_tree, i, forest.estimators_[i], Kcomp_list[i]): i
            for i in range(total_trees)
        }
        for future in concurrent.futures.as_completed(futures):
            tree_idx = futures[future]
            encrypted_thresholds[tree_idx] = future.result()

            # Print progress updates every 10%
            if (tree_idx + 1) % (total_trees // 10) == 0:
                percent_done = ((tree_idx + 1) / total_trees) * 100
                print(f"✅ Encryption Progress: {percent_done:.0f}% completed...")

    return encrypted_thresholds

# 📌 Apply encryption with parallel execution
encrypted_thresholds_per_tree = encrypt_thresholds_parallel(rf_model, Kcomp_list)

print("✅ Threshold encryption completed.")
print(f"Example encrypted thresholds from first tree: {encrypted_thresholds_per_tree[0][:5]}")


🔒 Re-encrypting Decision Tree Thresholds with OP-NTRU (Optimized)...
✅ Encryption Progress: 20% completed...
✅ Encryption Progress: 10% completed...
✅ Encryption Progress: 30% completed...
✅ Encryption Progress: 40% completed...
✅ Encryption Progress: 50% completed...
✅ Encryption Progress: 60% completed...
✅ Encryption Progress: 70% completed...
✅ Encryption Progress: 80% completed...
✅ Encryption Progress: 90% completed...
✅ Encryption Progress: 100% completed...
✅ Threshold encryption completed.
Example encrypted thresholds from first tree: [1285487, 956557, 480588, 13634530, 4459109]


In [7]:
# 📌 Step 5.1: Secure OP-NTRU Encrypted Comparison with Randomized Encryption
def op_ntru_compare(x_enc, t_enc):
    """
    Securely compares an encrypted input feature with an encrypted threshold.

    Args:
        x_enc: Encrypted input feature (Bob's input).
        t_enc: Encrypted threshold (Alice's decision tree threshold).

    Returns:
        Boolean indicating if x_enc <= t_enc (True means go left, False means go right).
    """
    if x_enc == t_enc:
        print(f"⚠️ Warning: Encrypted values are equal ({x_enc} == {t_enc}) — check encryption scaling.")

    return x_enc <= t_enc  # OP-NTRU should preserve order

# 📌 Step 5.2: Secure Decision Tree Inference with Proper Feature Encryption
def encrypted_tree_inference(tree, X_enc, tree_idx):
    """
    Performs secure inference on a single decision tree using encrypted thresholds.

    Args:
        tree: A trained DecisionTreeClassifier.
        X_enc: Encrypted feature vector (Bob's encrypted input).
        tree_idx: The index of the tree in the Random Forest.

    Returns:
        The encrypted predicted label.
    """
    tree_ = tree.tree_
    node = 0  # Start from root

    while tree_.feature[node] != _tree.TREE_UNDEFINED:  # Traverse tree
        feature_idx = tree_.feature[node]

        # Handle cases where encrypted threshold is missing
        if node >= len(encrypted_thresholds_per_tree[tree_idx]):
            print(f"⚠️ Warning: Missing encrypted threshold for tree {tree_idx}, node {node}")
            break  # Prevent accessing out-of-range data

        threshold_enc = encrypted_thresholds_per_tree[tree_idx][node]

        # Secure comparison (compare encrypted input feature with encrypted threshold)
        if threshold_enc is not None and op_ntru_compare(X_enc[feature_idx], threshold_enc):
            node = tree_.children_left[node]  # Go left
        else:
            node = tree_.children_right[node]  # Go right

    # Return the final encrypted class prediction
    return tree_.value[node].argmax()

# 📌 Step 5.3: Secure Inference on Random Forest with Parallel Execution
def encrypted_forest_inference(forest, X_enc):
    """
    Performs secure inference on an entire Random Forest using encrypted thresholds.

    Args:
        forest: Trained RandomForestClassifier.
        X_enc: Encrypted feature vector (Bob's encrypted input).

    Returns:
        List of encrypted predictions from each tree.
    """
    encrypted_predictions = []

    def process_tree(i, tree):
        try:
            return encrypted_tree_inference(tree, X_enc, i)
        except IndexError:
            print(f"⚠️ Skipping Tree {i} due to missing encrypted thresholds.")
            return None

    # Use parallel processing for tree traversal
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_tree, range(len(forest.estimators_)), forest.estimators_))

    # Filter out None values
    encrypted_predictions = [pred for pred in results if pred is not None]

    return encrypted_predictions

# 📌 Step 5.4: Encrypt Bob's Feature Vector Correctly
print("🔐 Encrypting Bob's Input Feature Vector...")

# ✅ Fix: Use different encryption keys for different features
X_test_encrypted = np.array([
    op_ntru_encrypt(x, Ktree_list[i % len(Ktree_list)])  # Use different keys for each feature
    for i, x in enumerate(X_test_resized[0])
])

# ✅ Fix: Check if encryption produces unique values
print(f"🔒 Encrypted Input Features: {X_test_encrypted[:5]} (first 5 features)")

# 📌 Run Secure Inference on the Encrypted Random Forest
print("🧠 Running Secure Inference on Encrypted Random Forest...")
encrypted_predictions = encrypted_forest_inference(rf_model, X_test_encrypted)

# Print encrypted results
print(f"🔒 Encrypted Predictions from Trees: {encrypted_predictions[:5]} (first 5 trees)")


🔐 Encrypting Bob's Input Feature Vector...
🔒 Encrypted Input Features: [50707 91744 20407 37568 50093] (first 5 features)
🧠 Running Secure Inference on Encrypted Random Forest...
🔒 Encrypted Predictions from Trees: [np.int64(7), np.int64(7), np.int64(7), np.int64(7), np.int64(7)] (first 5 trees)


In [8]:
import math
import os
import concurrent.futures
import numpy as np
from collections import Counter, defaultdict
from pyope.ope import OPE
import hashlib
import random

# -----------------------------------------------------------------------------
# 📌 Step 6.0: Ensure `Kfinal` is Generated Only Once
# -----------------------------------------------------------------------------
def generate_ope_key():
    """
    Generates an OPE encryption key that remains consistent across encryption and decryption.

    Returns:
        An OPE encryption object.
    """
    key_seed = "Kfinal_secure_seed"  # Ensure consistent seed
    key_hash = hashlib.sha512(key_seed.encode()).digest()[:32]  # Hash key to get 256-bit key
    return OPE(key_hash)

# ✅ Ensure `Kfinal` is generated only once and remains the same
if 'Kfinal' not in globals():
    Kfinal = generate_ope_key()
    print(f"✅ Generated a valid OPE `Kfinal` key: {hash(str(Kfinal))}")
else:
    print(f"🔎 Debug: Using existing Kfinal with hash: {hash(str(Kfinal))}")

# -----------------------------------------------------------------------------
# 📌 Step 6.1: Proxy Re-Encryption (Ensuring Kfinal Consistency)
# -----------------------------------------------------------------------------
def proxy_re_encrypt(encrypted_predictions, Kfinal):
    """
    Re-encrypts predictions from different trees into a common format for Bob.

    Args:
        encrypted_predictions: List of encrypted predictions from different trees.
        Kfinal: The final encryption key for majority voting.

    Returns:
        List of re-encrypted predictions.
    """
    if not isinstance(Kfinal, OPE):
        raise TypeError("❌ Kfinal must be an OPE encryption key, not bytes!")

    # ✅ Track errors instead of printing them all
    error_summary = defaultdict(int)
    re_encrypted_predictions = []

    for pred in encrypted_predictions:
        try:
            re_enc = Kfinal.encrypt(int(pred) + random.randint(1, 10))  # Apply random noise
            re_encrypted_predictions.append(re_enc)
        except Exception as e:
            error_summary[str(e)] += 1  # Track errors

    # ✅ Print summary of errors only
    if error_summary:
        print("⚠️ **Re-Encryption Issues Detected:**")
        for err, count in error_summary.items():
            print(f"⚠️ {count} occurrences of: {err}")

    return re_encrypted_predictions

# -----------------------------------------------------------------------------
# 📌 Step 6.2: Secure Majority Voting with Proxy Re-Encryption
# -----------------------------------------------------------------------------
def secure_majority_voting(encrypted_predictions, Kfinal):
    """
    Aggregates encrypted predictions from multiple trees using majority voting.

    Args:
        encrypted_predictions: List of encrypted predictions from the Random Forest.
        Kfinal: The final encryption key for secure voting.

    Returns:
        Encrypted final prediction.
    """
    if not encrypted_predictions:
        raise ValueError("⚠️ No predictions received. Check encrypted inference.")

    # 📌 Fix: Convert predictions to integers & filter invalid values
    encrypted_predictions = [int(pred) for pred in encrypted_predictions if pred is not None]

    if not encrypted_predictions:
        raise ValueError("⚠️ No valid predictions available after filtering.")

    # ✅ Re-encrypt predictions for consistency
    re_encrypted_predictions = proxy_re_encrypt(encrypted_predictions, Kfinal)

    # ✅ Count occurrences of each prediction
    prediction_counts = Counter(re_encrypted_predictions)

    # ✅ Select the most common prediction (majority vote)
    final_encrypted_prediction = max(prediction_counts, key=prediction_counts.get)

    return final_encrypted_prediction

# -----------------------------------------------------------------------------
# 📌 Step 6.3: Optimize Secure Random Forest Inference with Parallel Processing
# -----------------------------------------------------------------------------
def encrypted_forest_inference(forest, X_enc):
    """
    Performs secure inference on an entire Random Forest using encrypted thresholds.

    Args:
        forest: Trained RandomForestClassifier.
        X_enc: Encrypted feature vector.

    Returns:
        List of encrypted predictions from each tree.
    """
    encrypted_predictions = [None] * len(forest.estimators_)

    def process_tree(i, tree):
        try:
            return encrypted_tree_inference(tree, X_enc, i)
        except IndexError:
            return None  # Skip tree if encrypted threshold is missing

    # ✅ Use parallel processing for tree traversal
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_tree, i, tree): i for i, tree in enumerate(forest.estimators_)}
        
        for future in concurrent.futures.as_completed(futures):
            tree_idx = futures[future]
            try:
                encrypted_predictions[tree_idx] = future.result()
            except Exception as e:
                print(f"⚠️ Error processing Tree {tree_idx}: {e}")

    # ✅ Filter out None values
    encrypted_predictions = [pred for pred in encrypted_predictions if pred is not None]

    return encrypted_predictions

# -----------------------------------------------------------------------------
# 📌 Run Secure Random Forest Inference
# -----------------------------------------------------------------------------
print("🧠 Running Secure Inference on Encrypted Random Forest...")
encrypted_predictions = encrypted_forest_inference(rf_model, X_test_encrypted)

# ✅ **Only show progress at key checkpoints** to prevent log flooding
print(f"✅ Completed inference on {len(encrypted_predictions)} trees.")

# -----------------------------------------------------------------------------
# 📌 Ensure `final_encrypted_prediction` is Stored Before Step 7
# -----------------------------------------------------------------------------
print("🗳️ Running Secure Majority Voting with Proxy Re-Encryption...")
final_encrypted_prediction = secure_majority_voting(encrypted_predictions, Kfinal)

print(f"🔒 Final Encrypted Prediction: {final_encrypted_prediction}")


✅ Generated a valid OPE `Kfinal` key: 5858925932499026038
🧠 Running Secure Inference on Encrypted Random Forest...
✅ Completed inference on 100 trees.
🗳️ Running Secure Majority Voting with Proxy Re-Encryption...
🔒 Final Encrypted Prediction: 659859


In [9]:
import math
import os
from pyope.ope import OPE
import hashlib

# -----------------------------------------------------------------------------
# 📌 Step 7.1: Ensure `Kfinal` is Generated as a Valid OPE Key
# -----------------------------------------------------------------------------
def generate_ope_key():
    """
    Generates an OPE encryption key that is valid for OP-NTRU decryption.

    Returns:
        An OPE encryption object.
    """
    key_seed = "Kfinal_secure_seed"  # Ensure consistent seed
    hashed_key = hashlib.sha512(key_seed.encode()).digest()[:32]  # Hash key for security
    return OPE(hashed_key)  # Return a valid OPE encryption key

# ✅ Ensure `Kfinal` exists and remains the same key used for encryption
if 'Kfinal' not in globals():
    raise ValueError("❌ Kfinal is missing! Ensure it is the same key used for encryption.")

# Print only ONE debug message for key validation
print(f"🔎 Debug: Hash of Kfinal before decryption: {hash(str(Kfinal))}")

# -----------------------------------------------------------------------------
# 📌 Step 7.2: Validate Final Encrypted Prediction Before Decryption
# -----------------------------------------------------------------------------

# ✅ Step 2: Ensure `final_encrypted_prediction` exists
if 'final_encrypted_prediction' not in locals() or final_encrypted_prediction is None:
    print("⚠️ `final_encrypted_prediction` not found, recalculating from Step 6...")
    final_encrypted_prediction = secure_majority_voting(encrypted_predictions, Kfinal)
    print(f"🔒 Recomputed Final Encrypted Prediction: {final_encrypted_prediction}")

# ✅ Step 2: Ensure `final_encrypted_prediction` is an integer
if not isinstance(final_encrypted_prediction, int):
    raise TypeError(f"❌ Error: Expected `final_encrypted_prediction` to be an integer, but got {type(final_encrypted_prediction)}.")

# ✅ Step 2: Ensure `final_encrypted_prediction` is within a valid numeric range
if final_encrypted_prediction < 0 or final_encrypted_prediction > 2**20:  # Adjust based on expected range
    raise ValueError(f"❌ Error: `final_encrypted_prediction` ({final_encrypted_prediction}) is outside expected range!")

# -----------------------------------------------------------------------------
# 📌 Step 7.3: Define OP-NTRU Decryption Function with Optimized Debugging
# -----------------------------------------------------------------------------
def op_ntru_decrypt(encrypted_value, ope_key, scale_factor=1000):
    """
    Decrypts an encrypted numerical value using OPE.

    Args:
        encrypted_value: The encrypted prediction.
        ope_key: The OPE decryption key.
        scale_factor: The factor used during encryption.

    Returns:
        Decrypted numerical value (original class label).
    """
    if not isinstance(ope_key, OPE):
        raise TypeError("Expected an OPE encryption object, but got a different type.")

    try:
        decrypted_value = ope_key.decrypt(encrypted_value)  # Use OPE decryption
    except Exception as e:
        print(f"❌ Decryption Error: {e} - Possible encryption mismatch with Kfinal!")
        return 0  # Default to class '0' if decryption fails

    # ✅ Improved Debugging: Print only essential decryption details

    # ✅ Adjust scaling logic to prevent flooring to zero
    if decrypted_value < scale_factor:
        scaled_value = decrypted_value  # Keep original value if already small
    else:
        scaled_value = decrypted_value // scale_factor  # Apply scaling only when necessary

    return max(0, min(scaled_value, 9))  # Ensure valid class label (0-9)

# -----------------------------------------------------------------------------
# 📌 Step 7.4: Bob Decrypts the Final Prediction
# -----------------------------------------------------------------------------
print(f"🔎 Debug: Hash of Kfinal before decryption: {hash(str(Kfinal))}")

# ✅ Ensure `final_encrypted_prediction` is within valid OPE range
ope_range = Kfinal.out_range
if final_encrypted_prediction < ope_range.start or final_encrypted_prediction > ope_range.end:
    print(f"❌ Decryption Error: {final_encrypted_prediction} is outside valid OPE range!")
    final_prediction = 0
else:
    final_prediction = op_ntru_decrypt(final_encrypted_prediction, Kfinal)

# ✅ Minimal Debugging Output Before Rescaling
print(f"🔎 Debug: Final decrypted prediction BEFORE rescaling: {final_prediction}")

# -----------------------------------------------------------------------------
# 📌 Step 7.5: Improved Rescaling of Decrypted Prediction
# -----------------------------------------------------------------------------
def rescale_decrypted_value(value):
    """
    Rescales the decrypted prediction to match the expected class label range (0-9).

    Args:
        value: The decrypted numerical value.

    Returns:
        A rescaled class label (integer between 0-9).
    """
    if value >= 10:
        return 9  # Cap values at 9
    if value < 0:
        return 0  # Floor negative values to 0
    return int(value)  # Keep valid values (0-9) unchanged

# ✅ Apply the fix
final_prediction_rescaled = rescale_decrypted_value(final_prediction)
print(f"🎯 Corrected Decrypted Final Prediction (Class Label): {final_prediction_rescaled}")


🔎 Debug: Hash of Kfinal before decryption: 5858925932499026038
🔎 Debug: Hash of Kfinal before decryption: 5858925932499026038
🔎 Debug: Final decrypted prediction BEFORE rescaling: 9
🎯 Corrected Decrypted Final Prediction (Class Label): 9


In [10]:
import time
import numpy as np
import concurrent.futures
from collections import defaultdict
from sklearn.metrics import accuracy_score

# 📌 Step 1: Evaluate Plaintext Model Accuracy & Runtime
print("\n⏳ Evaluating Plaintext Model...")
start_time_plain = time.time()
plaintext_predictions = rf_model.predict(X_test_resized)  # Predict on plaintext data
end_time_plain = time.time()

# Compute accuracy
plaintext_accuracy = accuracy_score(y_test, plaintext_predictions)
plaintext_runtime = end_time_plain - start_time_plain
print(f"✅ Plaintext Model Accuracy: {plaintext_accuracy:.4f}")
print(f"⏱️ Plaintext Inference Runtime: {plaintext_runtime:.4f} seconds")

# 📌 Step 2: Evaluate Encrypted Model Accuracy & Runtime
correct_predictions = 0
total_samples = len(X_test_resized)

print("\n⏳ Evaluating Encrypted Model...")
start_time_encrypted = time.time()

# ✅ Dictionary to track errors
error_summary = defaultdict(int)

# ✅ Encrypt the entire dataset at once
def encrypt_full_dataset(X_test, Ktree):
    """Encrypts all test samples at once using NumPy vectorization."""
    return np.array([[op_ntru_encrypt(x, Ktree) for x in sample] for sample in X_test])

X_test_encrypted = encrypt_full_dataset(X_test_resized, Ktree_list[0])  # 🔥 Encrypt everything first

# ✅ Process secure inference in parallel
def process_encrypted_batch(batch_indices):
    try:
        batch_X = X_test_encrypted[batch_indices]

        # **Run Secure Inference**
        encrypted_predictions = [encrypted_forest_inference(rf_model, X_enc) for X_enc in batch_X]

        # **Secure Majority Voting & Decryption**
        final_predictions = [op_ntru_decrypt(secure_majority_voting(enc_pred, Kfinal), Kfinal) for enc_pred in encrypted_predictions]

        # **Rescale and Compare**
        rescaled_predictions = [rescale_decrypted_value(pred) for pred in final_predictions]
        return [rescaled_predictions[i] == y_test[idx] for i, idx in enumerate(batch_indices)]

    except Exception as e:
        error_summary[str(e)] += 1  # ✅ Track errors
        return [False] * len(batch_indices)

# ✅ Run Secure Inference in Large Batches
batch_size = 100  # 🔥 Large batch size for efficiency
batched_indices = [list(range(i, min(i + batch_size, total_samples))) for i in range(0, total_samples, batch_size)]

results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_batch = {executor.submit(process_encrypted_batch, batch): batch for batch in batched_indices}

    for i, future in enumerate(concurrent.futures.as_completed(future_to_batch)):
        results.append(future.result())  # Store results
        if i % 5 == 0:  # Print progress every 5 batches
            print(f"✅ Progress: {i}/{len(batched_indices)} batches completed...")

# Flatten results
correct_predictions = sum([sum(batch) for batch in results])
encrypted_accuracy = correct_predictions / total_samples
end_time_encrypted = time.time()

# Compute runtime
encrypted_runtime = end_time_encrypted - start_time_encrypted

print(f"✅ Encrypted Model Accuracy: {encrypted_accuracy:.4f}")
print(f"⏱️ Encrypted Inference Runtime: {encrypted_runtime:.4f} seconds")

# 📌 Step 3: Compare Performance Impact
print("\n🔎 **Performance Impact Analysis**")
print(f"🔹 Accuracy Drop Due to Encryption: {plaintext_accuracy - encrypted_accuracy:.4f}")
print(f"🔹 Runtime Overhead Due to Encryption: {encrypted_runtime / plaintext_runtime:.2f}x slower")

# 📌 Step 4: **Final Debug Summary**
if error_summary:
    print("\n⚠️ **Summary of Issues Encountered:**")
    for error, count in error_summary.items():
        print(f"⚠️ {count} occurrences of: {error}")
    print(f"⚠️ Total unique errors: {len(error_summary)}")
else:
    print("\n✅ No critical issues detected during encrypted evaluation.")



⏳ Evaluating Plaintext Model...
✅ Plaintext Model Accuracy: 0.9693
⏱️ Plaintext Inference Runtime: 0.4461 seconds

⏳ Evaluating Encrypted Model...


KeyboardInterrupt: 

In [ ]:
print(f"Plaintext Model Accuracy: {plaintext_accuracy:.4f}")
print(f"Encrypted Model Accuracy: {encrypted_accuracy:.4f}")

NameError: name 'plaintext_accuracy' is not defined